# Second exercise

#### Importing necessery packages

In [1]:
import pysam

#### Creating a function for decoding read flags

In [2]:
def decode_read_flags(hex_value):
    flag_descriptions = {
        0x1: "read paired",
        0x2: "read mapped in proper pair",
        0x4: "read unmapped",
        0x8: "mate unmapped",
        0x10: "read reverse strand",
        0x20: "mate reverse strand",
        0x40: "first in pair",
        0x80: "second in pair",
        0x100: "not primary alignment",
        0x200: "read fails platform/vendor quality checks",
        0x400: "read is PCR or optical duplicate",
        0x800: "supplementary alignment"
    }

    decoded_flags = []
    for flag_value, description in flag_descriptions.items():
        if hex_value & flag_value:
            decoded_flags.append(description)

    return decoded_flags

#### Creating the alignment file from the .bam file

In [4]:
alignment_file = pysam.AlignmentFile("./merged-tumor.bam", "rb")

#### Inspecting the first read, its fields and the flag

In [5]:
first_read = next(alignment_file)

print("##### ANALYSIS OF THE FIRST READ #####\n")
for key, value in first_read.to_dict().items():
	print(str(key) + " : " + str(value))
	
print("\nThe first read flag is: " + str(first_read.flag) + ", which stands for: " + str(decode_read_flags(first_read.flag)))

##### ANALYSIS OF THE FIRST READ #####

name : C0HVYACXX120402:7:1207:5722:57044
flag : 1187
ref_name : 21
ref_pos : 9483249
map_quality : 27
cigar : 76M
next_ref_name : =
next_ref_pos : 9483382
length : 209
seq : TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG
qual : ==<>@?@@>@D?>@C?>8JA5?>CC>?@???BB;H-:4A?D=DB8BDE?GBJCD@B8??E<A>CDJB@B>AA@@@C
tags : ['XA:Z:GL000217.1,-110754,76M,1;', 'BD:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'MD:Z:76', 'RG:Z:1', 'BI:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'NM:i:0', 'MQ:i:27', 'AS:i:76', 'XS:i:71']

The first read flag is: 1187, which stands for: ['read paired', 'read mapped in proper pair', 'mate reverse strand', 'second in pair', 'read is PCR or optical duplicate']


#### Calculating the average mapping quality of all the reads

In [6]:
total_reads = 0
total_unmapped = 0
total_zero_quality = 0
sum_mapping_quality = 0

In [7]:
for read in alignment_file:
    if read.flag & 0x0004:
        total_unmapped += 1
    total_reads += 1
    
    if read.mapping_quality == 0:
        total_zero_quality += 1
    
    sum_mapping_quality += read.mapping_quality

#### Separating the averages of all and only non zero mapping quality reads

In [8]:
avg_mapping_quality_all = sum_mapping_quality / total_reads
avg_mapping_quality_filtered = sum_mapping_quality / (total_reads - total_zero_quality)

#### Printing the results

In [9]:
print("\n##### ANALYSIS OF ALL READS #####\n")
print(f"Total number of reads: {total_reads}")
print(f"Total number of unmapped reads: {total_unmapped}")
print(f"Total number of zero mapping quality reads: {total_zero_quality}")
print(f"Average mapping quality of all reads: {avg_mapping_quality_all}")
print(f"Average mapping quality of non zero mapping quality reads: {avg_mapping_quality_filtered}")


##### ANALYSIS OF ALL READS #####

Total number of reads: 2921628
Total number of unmapped reads: 17765
Total number of zero mapping quality reads: 126628
Average mapping quality of all reads: 55.91380148328261
Average mapping quality of non zero mapping quality reads: 58.446986762075134


#### Closing the alignement file

In [10]:
alignment_file.close()